In [129]:
import pandas as pd
import numpy as np
import geopandas as gp
import pickle
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
#from scipy import ols
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [130]:
processed = pd.read_csv('processed_data/master_merged.csv')

In [131]:
#Run a Correlation Matrix

processed.corr()

,age,bldg_class_A0,bldg_class_A1,bldg_class_A2,bldg_class_A3,bldg_class_A4,bldg_class_A5,bldg_class_A6,bldg_class_A7,bldg_class_A8,...,DOB_permit_EQ,DOB_permit_EW,DOB_permit_FO,DOB_permit_NB,DOB_permit_PL,DOB_permit_SG,DOB_permit_GAS,DOB_permit_OIL,POPULATION,AREA
age,1.000000,-0.128342,-0.263229,-0.454154,-0.131934,0.406617,-0.593104,-0.070372,0.022584,0.769455,...,0.229044,0.267912,-0.368527,-0.552982,-0.204021,-0.019259,0.109723,0.365677,0.285288,-0.301449
bldg_class_A0,-0.128342,1.000000,0.327357,0.444521,0.043769,-0.236724,-0.089278,-0.138400,0.006175,NaN,...,-0.073172,-0.192815,0.344923,-0.163672,0.134392,0.058985,-0.185018,-0.218928,-0.382438,-0.139605
bldg_class_A1,-0.263229,0.327357,1.000000,0.397142,0.206272,-0.332296,0.142390,0.135403,-0.037546,-1.000000,...,-0.267366,-0.492009,0.405906,0.208846,0.466488,0.030824,-0.401170,-0.480359,-0.431054,0.111019
bldg_class_A2,-0.454154,0.444521,0.397142,1.000000,0.006818,-0.197719,0.033392,0.025261,0.188307,-0.768357,...,-0.302293,-0.422769,0.434848,0.129162,0.419650,-0.010362,-0.313218,-0.459441,-0.480934,0.083536
bldg_class_A3,-0.131934,0.043769,0.206272,0.006818,1.000000,-0.100400,0.085931,-0.133306,0.303624,-1.000000,...,-0.134934,0.071425,0.006676,0.010048,-0.063380,-0.020456,-0.063266,0.028705,-0.196041,0.015888
bldg_class_A4,0.406617,-0.236724,-0.332296,-0.197719,-0.100400,1.000000,-0.240601,0.927396,0.537300,NaN,...,-0.154525,0.398323,-0.364542,-0.333281,-0.018072,-0.243455,0.229846,0.064466,-0.171831,-0.314772
bldg_class_A5,-0.593104,-0.089278,0.142390,0.033392,0.085931,-0.240601,1.000000,-0.151979,-0.289713,1.000000,...,-0.389673,-0.411220,0.312292,0.403861,0.400312,0.171296,-0.279556,-0.239172,-0.054973,0.396059
bldg_class_A6,-0.070372,-0.138400,0.135403,0.025261,-0.133306,0.927396,-0.151979,1.000000,0.874594,-1.000000,...,0.287880,-0.276209,-0.191346,0.245543,-0.076575,-0.426744,-0.162772,-0.116000,-0.135350,0.039828
bldg_class_A7,0.022584,0.006175,-0.037546,0.188307,0.303624,0.537300,-0.289713,0.874594,1.000000,NaN,...,-0.293575,0.308563,-0.218278,-0.222833,-0.011971,-0.284787,0.100817,-0.035690,-0.116897,-0.210012
bldg_class_A8,0.769455,NaN,-1.000000,-0.768357,-1.000000,NaN,1.000000,-1.000000,NaN,1.000000,...,0.942473,-0.892707,0.997036,-0.997860,0.786939,1.000000,NaN,-0.961304,-0.499326,-0.753872


In [132]:
#print processed['target.gas_incidents']

In [133]:
processed['gasperunit'] = processed['target.gas_incidents']/processed.total_units

In [147]:
#processed.replace([np.inf, -np.inf], np.nan)

In [149]:
processed = processed.fillna(0)

In [150]:
processed = processed.replace(np.inf, 0)

In [151]:
df = processed.drop(['gasperunit', 'target.gas_incidents', 'geometry'], axis=1)

In [152]:
colnames = df.columns.tolist()

In [153]:
#colnames

In [154]:
j = []
for i in range(len(colnames)):
    j.append('x'+str(i))
df.columns = j

In [155]:
#form = 'gasperunit ~ ' + ' + '.join(df.columns)

In [156]:
colnames[230]

'unit_area'

In [157]:
df[df.columns[230]].max()

1584455.75

In [161]:
#Statsmodels cannot handle the size of this dataset - use scipy below
#lm = smf.ols('gasperunit ~ '+ ' + '.join(df.columns), data = processed).fit()

In [164]:
lm = LinearRegression()
fit = lm.fit(df[df.columns], processed.gasperunit)